In [2]:
#!/usr/bin/env python3
# coding: utf-8
import sys
import pickle
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model,load_model

optuna_csv = sys.argv[1]

with open('training.pkl', 'rb') as f:
     train = pickle.load(f)
    
with open('testing.pkl', 'rb') as f:
     test = pickle.load(f)
        
with open('validation.pkl','rb') as f:
     val = pickle.load(f)
        
train_photos, train_labels = list(), list()
tp = list()
for file in train:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    train_photos.append(photo)
    train_labels.append(output)
train_photos = asarray(train_photos)
train_labels = asarray(train_labels)

test_photos, test_labels = list(), list()
for file in test:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    tp.append(photo)
    test_photos.append(photo)
    test_labels.append(output)
test_photos = asarray(test_photos)
test_labels = asarray(test_labels)

val_photos, val_labels = list(), list()
for file in val:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    val_photos.append(photo)
    val_labels.append(output)
val_photos = asarray(val_photos)
val_labels = asarray(val_labels)

train_photos =train_photos.astype('float32')
test_photos= test_photos.astype('float32')
val_photos =val_photos.astype('float32')
train_photos /= 255
val_photos /= 255
test_photos /= 255


from optkeras.optkeras import OptKeras
import optkeras
import pickle
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model,load_model
from keras.optimizers import RMSprop

import optuna
import os
import tensorflow as tf
checkpoint_path = "cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

nb_classes = 2
epochs=3
batch_size =2

vgg16_model = VGG16(weights = 'imagenet', include_top = False)
x = vgg16_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(nb_classes, activation = 'softmax')(x)
model = Model(input = vgg16_model.input, output = predictions)

for layer in vgg16_model.layers:
    layer.trainable = False
    
model.compile(optimizer = 'rmsprop',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True,
    period=1)
model.save_weights(checkpoint_path.format(epoch=0))
model_info = model.fit(x=train_photos, y=train_labels,batch_size=2 , epochs=epochs,callbacks=[cp_callback], 
                   verbose=1,validation_data=(val_photos,val_labels))

model.save('model_hpo.h5')

m= load_model("model_hpo.h5")
test_photos = test_photos.astype("float32")

results = m.evaluate(test_photos, test_labels, batch_size=1)


Using TensorFlow backend.
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:118: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Instructions for updating:
Use tf.cast instead.
Train on 432 samples, validate on 48 samples
Epoch 1/3


428/432 [============================>.] - ETA: 2:16 - loss: 0.9185 - acc: 0.500 - ETA: 1:42 - loss: 1.8639 - acc: 0.500 - ETA: 1:31 - loss: 1.4359 - acc: 0.666 - ETA: 1:23 - loss: 1.0812 - acc: 0.750 - ETA: 1:17 - loss: 0.8670 - acc: 0.800 - ETA: 1:14 - loss: 1.0761 - acc: 0.750 - ETA: 1:12 - loss: 1.0583 - acc: 0.642 - ETA: 1:10 - loss: 1.1688 - acc: 0.625 - ETA: 1:09 - loss: 1.1267 - acc: 0.611 - ETA: 1:08 - loss: 1.0537 - acc: 0.650 - ETA: 1:07 - loss: 1.1871 - acc: 0.590 - ETA: 1:06 - loss: 1.1606 - acc: 0.583 - ETA: 1:05 - loss: 1.1081 - acc: 0.615 - ETA: 1:04 - loss: 1.1821 - acc: 0.571 - ETA: 1:03 - loss: 1.1665 - acc: 0.566 - ETA: 1:02 - loss: 1.1515 - acc: 0.531 - ETA: 1:02 - loss: 1.2024 - acc: 0.500 - ETA: 1:02 - loss: 1.1897 - acc: 0.500 - ETA: 1:01 - loss: 1.1788 - acc: 0.473 - ETA: 1:00 - loss: 1.2398 - acc: 0.450 - ETA: 1:00 - loss: 1.2154 - acc: 0.452 - ETA: 59s - loss: 1.1933 - acc: 0.431 - ETA: 59s - loss: 1.1652 - acc: 0.45 - ETA: 58s - loss: 1.1344 - acc: 0.47 - ET

432/432 [==============================] - ETA: 0s - loss: 0.6746 - acc: 0.648 - 63s 145ms/step - loss: 0.6788 - acc: 0.6481 - val_loss: 0.5038 - val_acc: 0.7708

Epoch 00001: saving model to cp.ckpt
Epoch 2/3


430/432 [============================>.] - ETA: 53s - loss: 0.1737 - acc: 1.00 - ETA: 53s - loss: 0.0982 - acc: 1.00 - ETA: 53s - loss: 0.1905 - acc: 1.00 - ETA: 53s - loss: 0.5045 - acc: 0.87 - ETA: 52s - loss: 0.5238 - acc: 0.80 - ETA: 52s - loss: 0.5766 - acc: 0.75 - ETA: 52s - loss: 0.5409 - acc: 0.78 - ETA: 52s - loss: 0.5127 - acc: 0.81 - ETA: 52s - loss: 0.4934 - acc: 0.83 - ETA: 51s - loss: 0.4683 - acc: 0.85 - ETA: 51s - loss: 0.4777 - acc: 0.81 - ETA: 51s - loss: 0.4591 - acc: 0.83 - ETA: 51s - loss: 0.4454 - acc: 0.84 - ETA: 50s - loss: 0.4302 - acc: 0.85 - ETA: 50s - loss: 0.4121 - acc: 0.86 - ETA: 50s - loss: 0.3884 - acc: 0.87 - ETA: 50s - loss: 0.4986 - acc: 0.85 - ETA: 49s - loss: 0.4970 - acc: 0.83 - ETA: 49s - loss: 0.5041 - acc: 0.81 - ETA: 49s - loss: 0.5031 - acc: 0.80 - ETA: 49s - loss: 0.5442 - acc: 0.76 - ETA: 49s - loss: 0.5378 - acc: 0.77 - ETA: 48s - loss: 0.5302 - acc: 0.78 - ETA: 48s - loss: 0.5180 - acc: 0.79 - ETA: 48s - loss: 0.5048 - acc: 0.80 - ETA: 48

432/432 [==============================] - 62s 144ms/step - loss: 0.4286 - acc: 0.8079 - val_loss: 0.9003 - val_acc: 0.5625

Epoch 00002: saving model to cp.ckpt
Epoch 3/3


430/432 [============================>.] - ETA: 54s - loss: 0.9570 - acc: 0.50 - ETA: 56s - loss: 0.5607 - acc: 0.75 - ETA: 55s - loss: 0.5694 - acc: 0.66 - ETA: 54s - loss: 0.4738 - acc: 0.75 - ETA: 54s - loss: 0.3864 - acc: 0.80 - ETA: 54s - loss: 0.3485 - acc: 0.83 - ETA: 54s - loss: 0.3202 - acc: 0.85 - ETA: 54s - loss: 0.4418 - acc: 0.81 - ETA: 54s - loss: 0.4942 - acc: 0.77 - ETA: 53s - loss: 0.4630 - acc: 0.80 - ETA: 53s - loss: 0.4245 - acc: 0.81 - ETA: 52s - loss: 0.3915 - acc: 0.83 - ETA: 52s - loss: 0.3910 - acc: 0.84 - ETA: 52s - loss: 0.3965 - acc: 0.82 - ETA: 52s - loss: 0.3902 - acc: 0.83 - ETA: 52s - loss: 0.3710 - acc: 0.84 - ETA: 51s - loss: 0.3772 - acc: 0.82 - ETA: 51s - loss: 0.3570 - acc: 0.83 - ETA: 51s - loss: 0.3412 - acc: 0.84 - ETA: 50s - loss: 0.3589 - acc: 0.82 - ETA: 50s - loss: 0.3595 - acc: 0.80 - ETA: 50s - loss: 0.4034 - acc: 0.77 - ETA: 49s - loss: 0.4012 - acc: 0.78 - ETA: 49s - loss: 0.4388 - acc: 0.75 - ETA: 49s - loss: 0.4335 - acc: 0.76 - ETA: 49

432/432 [==============================] - 63s 146ms/step - loss: 0.3517 - acc: 0.8519 - val_loss: 0.3038 - val_acc: 0.8542

Epoch 00003: saving model to cp.ckpt
120/120 [==============================] - ETA: 24 - ETA: 19 - ETA: 18 - ETA: 17 - ETA: 16 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:

In [ ]:
# def save_checkpoint(study,model,optimizer,current_metrics, checkpoints_files, losses_dict,trial,model_name):
#     model_state_dict = {}

#     if model_name in PRETRAINED_MODELS.keys():
#         whole_state_dict = model.state_dict()
#         all_keys = list(whole_state_dict.keys())
#         classifier_keys = all_keys[PRETRAINED_MODELS[model_name]:]

#         for k,v in whole_state_dict.items():
#             if k in classifier_keys:
#                 model_state_dict[k] = v
#     else:
#         model_state_dict = model.state_dict()

#     filename='chckpnt_trial_' + str(trial.number) +'_epoch_'+ str(current_metrics[0])+"_"+ model_name +'.pth'
#     checkpoint = { 'trail': trial.number,
#                  'run_parameters': str(trial.params),
#                  'curr_epoch': current_metrics[0],
#                  'model_state_dict' : model_state_dict,
#                  'optimizer_state_dict': optimizer.state_dict(),
#                  'checkpoints_list': checkpoints_files,
#                  'train_loss': current_metrics[1],
#                  'test_loss': current_metrics[2],
#                  'test_accuracy': current_metrics[3],
#                  'losses_dict': losses_dict,
#                  'optuna_study':study
#     }

#     checkpoints_files.append(filename)
#     torch.save(checkpoint, filename)
#     return checkpoints_files



In [5]:
# model.load_weights(checkpoint_path)
# loss,acc = model.evaluate(test_photos,  test_labels, verbose=0)
# print("Restored model, accuracy: {:5.2f}%".format(100*acc))

In [ ]:
import argparse
optkeras.optkeras.get_trial_default = lambda: optuna.trial.FrozenTrial(
            None, None, None, None, None, None, None, None, None, None, None)
study_name = "CatsAndDogs" + '_Simple'
ok = OptKeras(study_name=study_name,
              monitor='val_acc',
              direction='maximize')
import joblib
MODEL = "vgg"
STUDY = None
N_TRIALS = 3


def sigterm_handler(signum, frame):
    hpo_monitor_sigterm()
    sys.exit(3)
    
def hpo_monitor(study, trial):
    joblib.dump(study,"study_checkpoint_" + MODEL + ".pkl")   
    
def objective(trial):
    nb_classes = 2
    epochs=3
    batch_size =2
    optimizer_options = ["RMSprop", "Adam", "SGD"]

    vgg16_model = VGG16(weights = 'imagenet', include_top = False)
    x = vgg16_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation=trial.suggest_categorical('activation', ['relu', 'linear']))(x)
    predictions = Dense(nb_classes, activation = 'softmax')(x)
    model = Model(input = vgg16_model.input, output = predictions)
    for layer in vgg16_model.layers:
        layer.trainable = False

    model.compile(optimizer = trial.suggest_categorical("optimizer", ["rmsprop", "Adam", "SGD"]),loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True,
    period=10)
    model.fit(x=train_photos, y=train_labels,batch_size=2 , epochs=epochs, callbacks=ok.callbacks(trial),
              verbose=ok.keras_verbose ,validation_data=(val_photos,val_labels))
    return ok.trial_best_value

checkpoint_file = "study_checkpoint_vgg.pkl"

try:
    STUDY = joblib.load(checkpoint_file)
    finished = len(STUDY.trials_dataframe())
except AttributeError:
    STUDY = OptKeras(study_name=study_name,monitor='val_acc',direction='maximize')
    finished = 1


if finished < N_TRIALS:  
    todo_trials = N_TRIALS - finished
    if todo_trials > 0 :
        STUDY.optimize(objective, n_trials=todo_trials, timeout=600, callbacks=[hpo_monitor])
    else:
        pass
else:
    STUDY = OptKeras(study_name=study_name,monitor='val_acc',direction='maximize')
    STUDY.optimize(objective, n_trials=N_TRIALS, timeout=600, callbacks=[hpo_monitor])
        

[I 2020-09-10 12:45:35,570] A new study created in memory with name: CatsAndDogs_Simple


[2020-09-10 12:45:35.578116] [OptKeras] Ready for optimization. (message printed as verbose is set to 1+)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:32: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [46]:
hpo_checkpoint_file = "study_checkpoint_" + MODEL + ".pkl"
k = joblib.load(hpo_checkpoint_file)
k.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_activation,params_optimizer,user_attrs__Datetime_epoch_begin,user_attrs__Datetime_epoch_end,user_attrs__Trial_num,user_attrs_acc,user_attrs_loss,user_attrs_val_acc,user_attrs_val_loss,state
0,0,0.875000,2020-09-10 10:38:04.364084,2020-09-10 10:41:17.425286,0 days 00:03:13.061202,linear,rmsprop,2020-09-10 10:40:10.220480,2020-09-10 10:41:17.036755,0,0.840278,0.371861,0.875000,0.281831,COMPLETE
1,1,0.729167,2020-09-10 10:41:17.433711,2020-09-10 10:44:40.778047,0 days 00:03:23.344336,relu,Adam,2020-09-10 10:43:33.169837,2020-09-10 10:44:40.773838,1,0.854167,0.336904,0.729167,0.509749,COMPLETE


In [48]:
import os
import pickle

if not os.path.isfile("checkpoint_vgg.pkl"):
    with open("checkpoint_vgg.pkl",'wb') as file:
        pickle.dump("0", file)
        
checkpoint_file = "checkpoint_vgg.pkl"

# try:
STUDY = joblib.load(checkpoint_file)
finished = len(STUDY.trials_dataframe())
# except FileNotFoundError:


AttributeError: 'str' object has no attribute 'trials_dataframe'

In [33]:
try:
    signal.signal(signal.SIGTERM, sigterm_handler)
    args     = parser.parse_args()
    N_TRIALS = args.trials[0]
    MODEL = "vgg16"
    hpo_checkpoint_file = "study_checkpoint_" + MODEL + ".pkl"
    create_study(hpo_checkpoint_file)

except Exception as e:
    pass
finally:
    hpo_monitor_sigterm("final_")

In [26]:
def hpo_monitor_sigterm(final_prefix = ""):
    joblib.dump(STUDY,final_prefix + "hpo_study_checkpoint_" + MODEL + ".pkl")
    
def main():
    global MODEL
    global EPOCHS
    global CHECKPOINTS_FILES_LIST
    global TAR_FILE
    global N_TRIALS

    try:
#         signal.signal(signal.SIGTERM, sigterm_handler)
        
        args     = parser.parse_args()
        N_TRIALS = args.trials[0]
        MODEL = "vgg16"
        hpo_checkpoint_file = "hpo_study_checkpoint_" + MODEL + ".pkl"
        create_study(hpo_checkpoint_file)

    except Exception as e:
    	pass
#     finally:
#         hpo_monitor_sigterm("final_")
#     return 0


main()

usage: ipykernel_launcher.py [-h] num_trials
ipykernel_launcher.py: error: argument num_trials: invalid int value: '/home/scitech/.local/share/jupyter/runtime/kernel-515b4e2f-033d-41a9-934f-471f4b3125d7.json'
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib64/python3.6/argparse.py", line 2294, in _get_value
    result = type_func(arg_string)
ValueError: invalid literal for int() with base 10: '/home/scitech/.local/share/jupyter/runtime/kernel-515b4e2f-033d-41a9-934f-471f4b3125d7.json'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib64/python3.6/argparse.py", line 1766, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "/usr/lib64/python3.6/argparse.py", line 1975, in _parse_known_args
    stop_index = consume_positionals(start_index)
  File "/usr/lib64/python3.6/argparse.py", line 1931, in consume_positionals
    take_action(action, args)
  File "/usr/lib64/python3.6/argparse.py", line 1824, in take_action
    argument_values = self._get_values(action, argument_strings)
  File "/usr/lib64/python3.6/argparse.py", line 2279, in _get_values
    value = [self._get_value(action, v) 

TypeError: object of type 'NoneType' has no len()

In [ ]:
finished = 0
while finished != N_TRIALS:
    STUDY = joblib.load("hpo_study_checkpoint_" + MODEL + ".pkl")
    todo_trials = N_TRIALS - len(STUDY.trials_dataframe())
    if todo_trials > 0 :
        STUDY.optimize(objective, n_trials=todo_trials, timeout=600, callbacks=[hpo_monitor])
        finished += 1
    else:
        pass
    
def hpo_monitor(study, trial):
    joblib.dump(study,"hpo_study_checkpoint_" + MODEL + ".pkl")
    
def create_study(hpo_checkpoint_file):
    global STUDY
    STUDY = joblib.load("hpo_study_checkpoint_" + MODEL + ".pkl")
    
    if STUDY == None:
        finished = 1
    else:
        finished = len(STUDY.trials_dataframe()

    if finished < N_Trials:
        STUDY = joblib.load("hpo_study_checkpoint_" + MODEL + ".pkl")
        todo_trials = N_TRIALS - finished)
        if todo_trials > 0 :
            STUDY.optimize(objective, n_trials=todo_trials, timeout=600, callbacks=[hpo_monitor])
        else:
            pass
    else:
#         STUDY = optuna.create_study(direction = 'maximize', study_name = MODEL)
        STUDY = OptKeras.create_study(study_name=MODEL,monitor='val_acc',direction='maximize')
        STUDY.optimize(objective, n_trials=N_TRIALS, timeout=600, callbacks=[hpo_monitor])

In [35]:
import neptune
import neptunecontrib.monitoring.optuna as opt_utils

neptune.init('jakub-czakon/blog-hpo')
neptune.create_experiment(name='optuna sweep')

monitor = opt_utils.NeptuneMonitor()
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, callbacks=[monitor])
opt_utils.log_study(study)

AttributeError: module 'neptune' has no attribute 'init'

In [34]:
!sudo pip3 install neptune-cli neptune-contrib

     |████████████████████████████████| 64 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 90 kB 3.9 MB/s eta 0:00:011
     |████████████████████████████████| 55 kB 4.0 MB/s  eta 0:00:01
     |████████████████████████████████| 127 kB 21.6 MB/s eta 0:00:01
     |████████████████████████████████| 138 kB 17.0 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 7.2 MB/s  eta 0:00:01


  Created wheel for neptune-contrib: filename=neptune_contrib-0+unknown-py3-none-any.whl size=77523 sha256=73ac3558ff876c5b3587cb9f04098980156c8246b385e61713b47519b4691b95
  Stored in directory: /root/.cache/pip/wheels/ba/13/05/e2a2aa31e5fd73a26536765a8a3014aa87b313ee141878f3c8
  Created wheel for neptune-client: filename=neptune_client-0.4.119-py2.py3-none-any.whl size=150018 sha256=c5ae1cf94101b404ac540fc930b306049506b890668f7b9a59121b935dcbe804
  Stored in directory: /root/.cache/pip/wheels/0c/89/27/eb0f1475ebb29351a79e017cc4e6b42607f77c17d8267a6578
  Created wheel for msgpack-python: filename=msgpack_python-0.5.6-cp36-cp36m-linux_x86_64.whl size=311832 sha256=d002832017063ec351c307d81c0de4521a8e3f7e39ba2c7317b2d8a17ea21537
  Stored in directory: /root/.cache/pip/wheels/d5/bc/14/cd44eeb567de86911dd5ca37e718e67044d03b6128eb80ef1b
  Created wheel for strict-rfc3339: filename=strict_rfc3339-0.7-py3-none-any.whl size=18119 sha256=796c236b9bba916489150a6d437d01498768215d5f67162f5cfdc490e

In [ ]:
import argparse
optkeras.optkeras.get_trial_default = lambda: optuna.trial.FrozenTrial(
            None, None, None, None, None, None, None, None, None, None, None)
study_name = "CatsAndDogs" + '_Simple'
ok = OptKeras(study_name=study_name,
              monitor='val_acc',
              direction='maximize')
import joblib
MODEL = "vgg"
CHECKPOINTS_FILES_LIST = []
STUDY = None
N_TRIALS = 5
# checkpoint_path = "cp-{epoch:04d}.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

parser = argparse.ArgumentParser(description='Binary Classification of Images with Different Architectures')
parser.add_argument('trials',  metavar='num_trials', type=int, nargs=1, help = "number of HPO trials")

def sigterm_handler(signum, frame):
    hpo_monitor_sigterm()
    sys.exit(3)
    
def hpo_monitor(study, trial):
    joblib.dump(study,"hpo_study_checkpoint_" + MODEL + ".pkl")
    
def hpo_monitor_sigterm(final_prefix = ""):
    joblib.dump(STUDY,final_prefix + "hpo_study_checkpoint_" + MODEL + ".pkl")
    
def objective(trial):
    nb_classes = 2
    epochs=3
    batch_size =2
    optimizer_options = ["RMSprop", "Adam", "SGD"]

    vgg16_model = VGG16(weights = 'imagenet', include_top = False)
    x = vgg16_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation=trial.suggest_categorical('activation', ['relu', 'linear']))(x)
    predictions = Dense(nb_classes, activation = 'softmax')(x)
    model = Model(input = vgg16_model.input, output = predictions)
    for layer in vgg16_model.layers:
        layer.trainable = False

    model.compile(optimizer = trial.suggest_categorical("optimizer", ["rmsprop", "Adam", "SGD"]),loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_weights_only=True,
    period=10)
    model.fit(x=train_photos, y=train_labels,batch_size=2 , epochs=epochs, callbacks=ok.callbacks(trial),
              verbose=ok.keras_verbose ,validation_data=(val_photos,val_labels))
    return ok.trial_best_value

def check_status_last_trial():
    global STUDY
    last_trial_index = len(STUDY.trials) -1
    last_trial_status = str(STUDY.trials[last_trial_index].state)
    
def create_study(finished,hpo_checkpoint_file):
    global STUDY
    if finished < N_TRIALS:
        STUDY = joblib.load("hpo_study_checkpoint_" + MODEL + ".pkl")
        todo_trials = N_TRIALS - len(STUDY.trials_dataframe())
        if todo_trials > 0 :
            STUDY.optimize(objective, n_trials=todo_trials, timeout=600, callbacks=[hpo_monitor])
            finished += 1
        else:
            pass
    else:
        STUDY = optuna.create_study(direction = 'maximize', study_name = MODEL)
        STUDY.optimize(objective, n_trials=N_TRIALS, timeout=600, callbacks=[hpo_monitor])
